# Análisis Integral para la Detección y Prevención de Fraudes en Tarjetas – Perspectiva del Emisor 

El objetivo principal de este proyecto es analizar las transacciones desde la perspectiva del emisor de tarjetas, con el propósito de identificar tendencias y detectar posibles comportamientos inusuales asociados al fraude. Para esto, se automatiza el cálculo de indicadores clave y KPIs relevantes, junto con la visualización de métricas útiles para el monitoreo y control operativo. Durante el trabajo, se identificarán variables de riesgo que permitan generar reglas en tiempo real (RT) para declinar transacciones fraudulentas, así como reglas cercanas a tiempo real (NRT) que faciliten el control y seguimiento de operaciones inusuales. Los resultados obtenidos servirán como base para diseñar recomendaciones y estrategias que fortalezcan la detección y prevención de fraudes, aumentando la capacidad de respuesta y la protección de las instituciones emisoras de tarjetas.

Cabe destacar que el uso de la Ciencia de Datos resulta esencial en este contexto, ya que permite transformar grandes volúmenes de información transaccional en conocimiento útil y accionable. Este enfoque no solo facilita la identificación temprana de riesgos y patrones sospechosos, sino que también potencia la toma de decisiones informadas, contribuyendo de manera decisiva a la gestión proactiva y eficiente de la prevención de fraudes en el sector financiero.

---

## Motivación y justificación 

- Automatizar la generación y reporte de información relevante para la toma de decisiones ágiles y efectivas en la detección y prevención de fraudes.
- Proveer una plataforma analítica robusta que sustente el diseño, priorización y ajuste de controles antifraude en RT/NRT, basada en evidencias y criterios medibles.
- Optimizar la eficiencia operativa, reduciendo pérdidas por fraude sin comprometer la satisfacción y experiencia del cliente.
- Garantizar la trazabilidad y objetividad en el manejo de datos, facilitando el desarrollo y seguimiento de modelos predictivos de machine learning.

---

## Objetivos 

- Desarrollar indicadores e informes de métricas clave de forma automatizada, como soporte a las decisiones estratégicas antifraude.
- Implementar y monitorear KPIs críticos para la evaluación continua de desempeño y riesgo.
- Analizar y segmentar el histórico de transacciones para descubrir patrones y comportamientos con alta propensión a fraude.
- Extraer y definir variables explicativas basadas en análisis estadístico y evidencia empírica.
- Construir y validar modelos de machine learning orientados a la detección temprana y la prevención eficaz de fraudes en tarjetas, bajo la perspectiva del emisor.

---

## Autor 

-Christian Javier Fernández Ocaña 

* Período de cobertura de datos: desde el segundo trimestre de 2023 hasta el primer trimestre de 2025

* Datos para uso educativo
---

# 1. Configuración del entorno

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns
from IPython.display import display
from sqlalchemy import create_engine
pd.set_option ('display.max_columns', None)
pd.set_option ('display.max_rows', None)


# 2. Carga de datos

In [2]:
# Conexión a la base de datos
usuario = "root"
password = '1993'
host = 'Localhost'
puerto = '3306'
base_datos = 'trx_emisor_sumulados_v1'

# Crear engine de conexión

engine = create_engine(f"mysql+pymysql://{usuario}:{password}@{host}:{puerto}/{base_datos}")

# Cargar la tabla

query = (
    "SELECT DE32_Cod_Adquiriente, DE42_Cod_Comercio, DE49_Cod_Moneda, DE61_13_Cod_Pais, ID_Usuario, DE13_Fecha, "
    "Id_Trx, Comercio_Alias, DE18_MCC, DE6_Monto_Dolar, DE4_Monto_Local, DE22_Modo_Entrada, Aut, "
    "DE25_Punto_Entrada, Regla, DE39_Respuesta_ISO, Id_Terminal_Hash, MTI, Cod_Resp, entry_mode, "
    "Desc_entry_mode, canal, Registro, IIN, ID_Tarjeta_Hash, Estado, BIN_TC_TD, dia_semana, Respuesta "
    "FROM tabla_trx_emisor "
    "WHERE BIN_TC_TD = 'TD'"
)

df_trx_emisor = df_trx_emisor = pd.read_sql(query, con=engine)


# 3. Inspección y arreglos preliminares

## Definición de tipos de datos

Para evitar errores durante el análisis, es fundamental asignar explícitamente el tipo de dato adecuado a cada variable del conjunto de datos, asegurando que las columnas categóricas, numéricas y de fechas estén correctamente tipificadas.

Por ejemplo, variables como códigos, identificadores y estados se definen como cadenas de texto (strings), mientras que los montos se convierten a tipo flotante para facilitar cálculos y segmentaciones posteriores.

---

### Contexto: Estándar ISO 8583

El análisis de transacciones con tarjetas se basa en el estándar internacional **ISO 8583**, un protocolo global que define la estructura y formato de mensajes para transacciones financieras originadas por tarjetas de crédito y débito. Este estándar es utilizado ampliamente en sistemas financiero y redes de pago para garantizar la interoperabilidad y seguridad en la comunicación de transacciones electrónicas.

---

### 🔑 Variables clave en este análisis

| Variable             | Descripción                                                                                       |
|----------------------|-------------------------------------------------------------------------------------------------|
| **IIN**              | Número de Identificación del Usuario o BIN, usado para clasificar la tarjeta.                    |
| **DE32_Cod_Adquiriente** | Código del adquirente que procesa la transacción (datos anonimizados).                      |
| **DE42_Cod_Comercio** | Código del comercio o establecimiento donde se realiza la transacción.                           |
| **DE61_13_Cod_Pais** | Código del país desde donde se origina la transacción.                                          |
| **ID_Usuario**       | Identificador único del usuario.                                                                 |
| **Comercio_Alias**   | Identificador pseudonimizado del establecimiento para proteger la privacidad.                     |
| **Canal**            | Indica el canal de consumo: tarjeta presente o no presente.                                     |
| **DE18_MCC**         | Giro o categoría del comercio según Merchant Category Code (MCC).                               |
| **Respuesta**        | Indica si la transacción fue autorizada o rechazada.                                            |
| **Fraude**           | Variable binaria: 1 si la transacción es fraudulenta, 0 si no.                                  |
| **DE6_Monto_Dolar**  | Monto de la transacción expresado en dólares estadounidenses.                                   |
| **DE4_Monto_Local**  | Monto de la transacción en moneda local de origen.                                              |
| **ID_Tarjeta_Hash**  | Identificador pseudonimizado que simula un número de tarjeta único.                             |

---

## Definir los tipos de datos de cada columna


In [4]:
# Definir los tipos de cada columnas
dtype_cols = {
    "INN":str,
    "DE32_Cod_Adquiriente": str,
    "DE42_Cod_Comercio": str,
    "DE49_Cod_Moneda": str,
    "DE61_13_Cod_Pais": str,
    "ID_Usuario": str,
    "Id_Trx": str,
    'Comercio_Alias': str,
    "DE18_MCC": str,
    "DE22_Modo_Entrada": str,
    "Aut": str,
    "Pais_Comercio": str,
    "DE25_Punto_Entrada": str,
    "Regla": str,
    "DE39_Respuesta_ISO": str,
    "Id_Terminal_Hash": str,
    "DE39_Terminal_Id": str,
    "Tipo_Moneda": str,
    "MTI": str,
    "Cod_Resp": str,
    "No_Referencia": str,
    "dia_semana": str,
    "entry_mode": str,
    "Desc_entry_mode": str,
    "canal": str,
    "Respuesta": str,
    "Registro": str,
    "Estado": str,
    "DE6_Monto_Dolar": float,
    "DE4_Monto_Local": float
}


# Forzar tipos de datos segùn lo definido

for col, tipo in dtype_cols.items():
    if col in df_trx_emisor.columns:
        df_trx_emisor[col] = df_trx_emisor[col].astype(tipo, errors='ignore')

## Definir rangos 

- Por montos de transacciòn 

In [5]:
# Definir rangos de montos por transacciòn

rangos = [0,1,10,100,300,1000, float('inf')]
labels = [
    "0-1",          # Validaciones de IIN 
    "1-10",         # Rangos de montos donde tambien se presentan pruebas
    "10-100",       # Pagos moderados
    "100-300",      # Montos medios
    "300-1000",     # Montos altos
    "+1000"         # Montos muy altos
]

# Crear la nueva columna de rangos

df_trx_emisor['rango_monto'] = pd.cut(
    df_trx_emisor['DE6_Monto_Dolar'],
    bins=rangos,
    labels = labels,
    right=True, 
    include_lowest= True

)

# Ver conteo por rango
df_trx_emisor['rango_monto'].value_counts().sort_index()




rango_monto
0-1         173832
1-10        408309
10-100      750055
100-300     109112
300-1000     21559
+1000          973
Name: count, dtype: int64

## Construcción de variables de tiempo y fraude
Para realizar un análisis temporal detallado y detectar patrones estacionales o por periodos, se generan variables derivadas a partir de la fecha original de cada transacción. Estas variables temporales permiten segmentar y observar la evolución de fraudes y transacciones en diferentes escalas de tiempo, como año, mes, día u hora.

También se crea una variable binaria para identificar si una transacción fue fraudulenta o no, lo cual es fundamental para el modelado y segmentación posterior.

---

### Variables temporales generadas:

- **year:** Año de la transacción.  
- **mes:** Mes del año.  
- **dia:** Día del mes.  
- **hora:** Hora del día.  
- **mes_año:** Periodo mes-año para análisis agregados mensuales.  
- **trimestre:** Periodo trimestral para detectar patrones estacionales.

In [6]:
#Crear variables Año, mes,dia, hora, dia semana
#Se generan variables temporales (año, mes, día, hora, trimestre, mes-año) a partir de la fecha de transacció y una variable binaria de fraude.
df_trx_emisor['year'] =df_trx_emisor['DE13_Fecha'].dt.year
df_trx_emisor['mes'] = df_trx_emisor['DE13_Fecha'].dt.month
df_trx_emisor['day'] = df_trx_emisor['DE13_Fecha'].dt.day
df_trx_emisor['hora'] = df_trx_emisor['DE13_Fecha'].dt.hour
df_trx_emisor['mes_year'] = df_trx_emisor['DE13_Fecha'].dt.to_period('M')
df_trx_emisor['trimestre'] = df_trx_emisor['DE13_Fecha'].dt.to_period('Q')

# Crear clumna binaria de fraude
df_trx_emisor['Fraude'] = np.where(df_trx_emisor['Estado'].str.lower() == 'f',1,0)          

## Construcción de rangos horarios

- Se segmentan las horas del día en diferentes franjas horarias, con el objetivo de clasificar y analizar los momentos en que ocurren las transacciones.

In [7]:
# Definir rangos por horarios

def rango_monto(hora):
    if 0 <= hora < 6:
        return "madrugada"
    elif 6 <= hora < 12:
        return "mañana"
    elif 12 <= hora < 18:
        return 'tarde'
    else:
        return 'noche'
    
df_trx_emisor['franja_horaria'] = df_trx_emisor['hora'].apply(rango_monto)

# ver conteo por rangos

df_trx_emisor['franja_horaria'].value_counts().sort_index()

franja_horaria
madrugada     69789
mañana       393677
noche        377235
tarde        623139
Name: count, dtype: int64

In [8]:
df_trx_emisor['Regla_nrt'] = np.where(
    (df_trx_emisor['Estado'] == 'None') | 
    ((df_trx_emisor['Estado'] == 'F') & (df_trx_emisor['Respuesta'] == 'AUTORIZADO')),
    'NO_ALERTA',
    'SI_ALERTA'
)

sialerta1 = df_trx_emisor.groupby('Regla_nrt')['Estado'].value_counts()
sialerta1

Regla_nrt  Estado
NO_ALERTA  None      1340445
           F            1196
SI_ALERTA  D          108947
           F            7933
           I            4838
           S             481
Name: count, dtype: int64

## Revisión de valores únicos y consistencia de datos

En esta sección se identifican y analizan los valores únicos presentes en las variables principales del dataset. Este paso es esencial para:

- Inspeccionar la consistencia y limpieza de los datos.
- Detectar posibles anomalías, duplicados o valores atípicos.
- Realizar una verificación rápida del estado general del conjunto de datos.
- Asegurar que las variables categóricas estén correctamente codificadas y sean coherentes para análisis posteriores.

Descripción general del proceso

Para cada variable clave, se realiza:

- Cálculo de la frecuencia absoluta de cada valor único.
- Cálculo de la proporción (%) respecto al total de observaciones para identificar categorías dominantes o raras.

 Esto facilita la toma de decisiones sobre limpieza, segmentación o consolidación de categorías.



In [9]:
# IIN
resumen_IIN =(
    df_trx_emisor['IIN']
    .value_counts()
    .reset_index()
    .rename(columns={'count':'frecuencia'})
 )
total_IIN = resumen_IIN['frecuencia'].sum()
resumen_IIN['%'] = (resumen_IIN['frecuencia']/ total_IIN *100).round(2)

# DE32_Cod_Adquiriente 
resumen_adquirente = (
    df_trx_emisor['DE32_Cod_Adquiriente']
    .value_counts()
    .reset_index()     
    .rename(columns={'count':'frecuencia'})
)
total_adquirente = resumen_adquirente['frecuencia'].sum()
resumen_adquirente['%'] = (resumen_adquirente['frecuencia']/ total_adquirente*100).round(2)


# DE42_Cod_Comercio
resumen_codcomercio = (
    df_trx_emisor['DE42_Cod_Comercio']
    .value_counts()
    .reset_index()
    .rename(columns = {'count':'frecuencia'})
)   
total_codcomercio = resumen_codcomercio['frecuencia'].sum()
resumen_codcomercio['%'] = (resumen_codcomercio['frecuencia']/total_codcomercio*100).round(2)


# DE61_13_Cod_Pais
resumen_pais =(
    df_trx_emisor['DE61_13_Cod_Pais']
    .value_counts()
    .reset_index()
    .rename(columns={'count':'frecuencia'})
)
total_pais = resumen_pais['frecuencia'].sum()
resumen_pais['%'] = (resumen_pais['frecuencia']/total_pais *100).round(2)

# Comercio_Alias
resumen_comercio = (
    df_trx_emisor['Comercio_Alias']
    .value_counts()
    .reset_index()
    .rename(columns={'count':'frecuencia'})
)
total_comercio = resumen_comercio['frecuencia'].sum()
resumen_comercio['%'] = (resumen_comercio['frecuencia']/ total_comercio *100).round(2)

# DE18_MCC
resumen_mcc = (
    df_trx_emisor['DE18_MCC']
    .value_counts()
    .reset_index()
    .rename(columns={'count':'frecuencia'})
)
total_mcc = resumen_mcc['frecuencia'].sum()
resumen_mcc['%'] = (resumen_mcc['frecuencia'] / (total_mcc)*100).round(2)

# DE6_Monto_Dolar
resumen_monto = (
    df_trx_emisor['DE6_Monto_Dolar']
    .value_counts()
    .reset_index()
    .rename(columns={'count':'frecuencia'})
)
total_monto = resumen_monto['frecuencia'].sum()
resumen_monto['%'] = ((resumen_monto['frecuencia'] / total_monto)*100 ).round(2)

# DE22_Modo_Entrada
resumen_entrada = (
    df_trx_emisor['DE22_Modo_Entrada']
    .value_counts()
    .reset_index()
    .rename(columns={'count':'frecuencia'})
)
total_entrada = resumen_entrada['frecuencia'].sum()
resumen_entrada['%'] = (resumen_entrada['frecuencia']/total_entrada * 100).round()

# Regla
resumen_regla = (
    df_trx_emisor['Regla']
    .value_counts()
    .reset_index()
    .rename(columns={'count':'frecuencia'}) 
)
total_regla = resumen_regla['frecuencia'].sum()
resumen_regla['%'] = (resumen_regla['frecuencia'] / total_regla *100).round(2)


# entry_mode
resumen_entry = (
    df_trx_emisor['entry_mode']
    .value_counts()
    .reset_index()
    .rename(columns={'count':'frecuencia'})
)
total_entry = resumen_entry['frecuencia'].sum()
resumen_entry['%'] = (resumen_entry['frecuencia']/total_entry * 100).round()

# Desc_entry_mode
resumen_desentry = (
    df_trx_emisor['Desc_entry_mode']
    .value_counts()
    .reset_index()
    .rename(columns={'count':'frecuencia'})
)
total_desentry = resumen_desentry['frecuencia'].sum()
resumen_desentry['%'] = (resumen_desentry['frecuencia']/total_desentry * 100).round()

# canal
resumen_canal = (
    df_trx_emisor['canal']
    .value_counts()
    .reset_index()
    .rename(columns={'count':'frecuencia'})
)
total_canal = resumen_canal['frecuencia'].sum()
resumen_canal['%'] = (resumen_canal['frecuencia'] / total_canal * 100).round(2)

# DE39_Respuesta_ISO
resumen_resp = (
    df_trx_emisor['DE39_Respuesta_ISO']
    .value_counts()
    .reset_index()
    .rename(columns={'count':'frecuencia'})
)
total_resp = resumen_resp['frecuencia'].sum()
resumen_resp['%'] = (resumen_resp['frecuencia']/ total_resp *100).round(2)

# Estado
resumen_estado = (
    df_trx_emisor['Estado']
    .value_counts()
    .reset_index()
    .rename(columns={'count':'frecuencia'})
)
total_estado = resumen_estado['frecuencia'].sum()
resumen_estado['%'] = (resumen_estado['frecuencia'] / total_estado * 100).round()

# dia_semana
resumen_dia_semana = (
    df_trx_emisor['dia_semana']
    .value_counts()
    .reset_index()
    .rename(columns={'count':'frecuencia'})
)
total_dia_semana = resumen_dia_semana['frecuencia'].sum()
resumen_dia_semana['%'] = (resumen_dia_semana['frecuencia'] / total_dia_semana * 100).round()

# rango_monto
resumen_rango_monto = (
    df_trx_emisor['rango_monto']
    .value_counts()
    .reset_index()
    .rename(columns={'count':'frecuencia'})
)
total_rango_monto = resumen_rango_monto['frecuencia'].sum()
resumen_rango_monto['%'] = (resumen_rango_monto['frecuencia'] / total_rango_monto * 100).round()

# year
resumen_year = (
    df_trx_emisor['year']
    .value_counts()
    .reset_index()
    .rename(columns={'count':'frecuencia'})
)
total_year = resumen_year['frecuencia'].sum()
resumen_year['%'] = (resumen_year['frecuencia'] / total_year * 100).round()


# hora
resumen_hora = (
    df_trx_emisor['hora']
    .value_counts()
    .reset_index()
    .rename(columns={'count':'frecuencia'})
)
total_hora = resumen_hora['frecuencia'].sum()
resumen_hora['%'] = (resumen_hora['frecuencia'] / total_hora * 100).round()

# franja_horaria
resumen_franja_horaria = (
    df_trx_emisor['franja_horaria']
    .value_counts()
    .reset_index()
    .rename(columns={'count':'frecuencia'})
)
total_franja_horaria = resumen_franja_horaria['frecuencia'].sum()
resumen_franja_horaria['%'] = (resumen_franja_horaria['frecuencia'] / total_franja_horaria * 100).round()

# Fraude
resumen_Fraude = (
    df_trx_emisor['Fraude']
    .value_counts()
    .reset_index()
    .rename(columns={'count':'frecuencia'})
)
total_Fraude = resumen_Fraude['frecuencia'].sum()
resumen_Fraude['%'] = (resumen_Fraude['frecuencia'] / total_Fraude * 100).round()

In [10]:
display(resumen_regla.head())

,Regla,frecuencia,%
0,0,1433663,97.94
1,175,4455,0.30
2,19,3202,0.22
3,24,2045,0.14
4,10,1629,0.11


In [11]:
display(resumen_IIN.head(),resumen_adquirente.head(3), resumen_codcomercio.head(2), resumen_pais.head(4),resumen_rango_monto.head(6),
        resumen_comercio.head(4),resumen_mcc.head(5), resumen_monto.head(),resumen_entrada.head(),resumen_regla.head(8),resumen_entry.head(5),
        resumen_desentry.head(), resumen_canal.head(), resumen_resp.head(), resumen_estado.head(),resumen_dia_semana.head(7), resumen_year.head(3),
        resumen_hora.head(10), resumen_Fraude.head(), resumen_franja_horaria.head())

,IIN,frecuencia,%
0,000001,1463840,100.0


,DE32_Cod_Adquiriente,frecuencia,%
0,ADQ_2925bbd5,520512,35.56
1,ADQ_5cd9439e,147280,10.06
2,ADQ_174af9a7,104421,7.13


,DE42_Cod_Comercio,frecuencia,%
0,COM_2925bbd5,520512,35.56
1,COM_645a8aca,173633,11.86


,DE61_13_Cod_Pais,frecuencia,%
0,218,1114344,76.12
1,840,249821,17.07
2,528,29447,2.01
3,372,21386,1.46


,rango_monto,frecuencia,%
0,10-100,750055,51.0
1,1-10,408309,28.0
2,0-1,173832,12.0
3,100-300,109112,7.0
4,300-1000,21559,1.0
5,+1000,973,0.0


...

,dia_semana,frecuencia,%
0,Friday,230164,16.0
1,Saturday,223693,15.0
2,Wednesday,211997,14.0
3,Monday,209044,14.0
4,Thursday,205127,14.0
5,Tuesday,199966,14.0
6,Sunday,183849,13.0


,year,frecuencia,%
0,2024,744372,51.0
1,2023,369239,25.0
2,2025,350229,24.0


,hora,frecuencia,%
0,12,109185,7.0
1,17,108376,7.0
2,13,108151,7.0
3,18,106629,7.0
4,11,100494,7.0
5,16,99611,7.0
6,14,99388,7.0
7,15,98428,7.0
8,10,92333,6.0
9,19,89323,6.0


,Fraude,frecuencia,%
0,0,1454711,99.0
1,1,9129,1.0


,franja_horaria,frecuencia,%
0,tarde,623139,43.0
1,mañana,393677,27.0
2,noche,377235,26.0
3,madrugada,69789,5.0


## Análisis de transacciones totales del DF

Durante la revisión inicial, se identificaron algunos valores extremadamente altos en las transacciones, que resultaron ser atípicos y probablemente datos mal capturados. Se detectaron tres registros que superaban significativamente los rangos esperados. Para evitar que estos valores sesgaran el análisis, fueron eliminados.

Tras este ajuste, el dataset presenta un comportamiento más consistente y representativo de la realidad operativa.

### Resumen del dataset:

- **Tamaño de la muestra:** 1,463,837 registros, representativos para el análisis.  
- **Montos (DE6_Monto_Dolar):** Principalmente bajos, con mediana en 15 USD y algunos valores atípicos que alcanzan hasta 17,150 USD. La distribución muestra una cola larga y sesgada.  
- **Variables temporales:** Años entre 2023 y 2025, con predominancia del 2024; mediana del mes en junio; día 15 y hora 14:00 h.  


### Conclusión

El dataset está dominado por transacciones de baja cuantía, con pocos valores extremos. La baja frecuencia de fraudes justifica el uso de técnicas específicas para tratar datasets desbalanceados. Esto permite un análisis temporal detallado y segmentado por monto.

In [12]:
# Se identificaron valores poco reales en las transacciones con tarjetas, específicamente un monto en dólares de $2,211,089,408.00. 
# Por esta razón, se procede a realizar una revisión más profunda con el fin de determinar las causas de este valor atipico en monto en dólares y el monto en moneda local.

pd.set_option('display.float_format','{:.2f}'.format)
df_trx_emisor.describe(include=[np.number])

,DE6_Monto_Dolar,DE4_Monto_Local,year,mes,day,hora,Fraude
count,1463840.00,1463840.00,1463840.00,1463840.00,1463840.00,1463840.00,1463840.00
mean,4571.74,5112.84,2023.99,6.61,15.28,13.83,0.01
std,3165336.04,3165472.34,0.70,3.56,8.74,4.88,0.08
min,0.00,0.00,2023.00,1.00,1.00,0.00,0.00
25%,5.15,5.00,2023.00,3.00,8.00,11.00,0.00
50%,15.42,16.00,2024.00,6.00,15.00,14.00,0.00
75%,45.00,48.00,2024.00,10.00,23.00,18.00,0.00
max,2211089408.00,2211089408.00,2025.00,12.00,31.00,23.00,1.00


In [13]:

# 1 Identificar el valor màximo identificado
valor_max = df_trx_emisor['DE6_Monto_Dolar'].max()
valor_max
# 2 Identificar filas con valores atipicos
df_max = df_trx_emisor[df_trx_emisor['DE6_Monto_Dolar']== valor_max ]
df_max


,DE32_Cod_Adquiriente,DE42_Cod_Comercio,DE49_Cod_Moneda,DE61_13_Cod_Pais,ID_Usuario,DE13_Fecha,Id_Trx,Comercio_Alias,DE18_MCC,DE6_Monto_Dolar,DE4_Monto_Local,DE22_Modo_Entrada,Aut,DE25_Punto_Entrada,Regla,DE39_Respuesta_ISO,Id_Terminal_Hash,MTI,Cod_Resp,entry_mode,Desc_entry_mode,canal,Registro,IIN,ID_Tarjeta_Hash,Estado,BIN_TC_TD,dia_semana,Respuesta,rango_monto,year,mes,day,hora,mes_year,trimestre,Fraude,franja_horaria,Regla_nrt
1232051,ADQ_7b28194e,COM_c9ce7f72,840,218,a78d03d2d6d2,2025-02-07 09:00:47,1486310,LC_9540c1090d,6011,2211089408.00,2211089408.00,0510,0,2252PRO11010P,0,51,TERM_fca396848a,0200,4007,05,Chip,CP,Compra,000001,e7591fbbaddb,None,TD,Friday,NO_AUTORIZADO,+1000,2025,2,7,9,2025-02,2025Q1,0,mañana,NO_ALERTA
1241525,ADQ_7b28194e,COM_c9ce7f72,840,218,a78d03d2d6d2,2025-02-11 12:28:17,1497210,LC_9540c1090d,6011,2211089408.00,2211089408.00,0510,0,2252PRO11010P,0,51,TERM_fca396848a,0200,4007,05,Chip,CP,Compra,000001,e7591fbbaddb,None,TD,Tuesday,NO_AUTORIZADO,+1000,2025,2,11,12,2025-02,2025Q1,0,tarde,NO_ALERTA
1241568,ADQ_7b28194e,COM_c9ce7f72,840,218,a78d03d2d6d2,2025-02-11 12:48:06,1497260,LC_9540c1090d,6011,2211089408.00,2211089408.00,0510,0,2252PRO11010P,0,51,TERM_fca396848a,0200,4007,05,Chip,CP,Compra,000001,e7591fbbaddb,None,TD,Tuesday,NO_AUTORIZADO,+1000,2025,2,11,12,2025-02,2025Q1,0,tarde,NO_ALERTA


- Durante la revisión de las transacciones, se identificó un valor atípico correspondiente al código MCC 6011 (ATM - Cajeros Automáticos). Este comportamiento sugiere la existencia de una posible anomalía en el registro de la operación.

- El hallazgo podría estar relacionado con errores de configuración en el cajero automático, los cuales habrían permitido procesar montos que exceden los límites normales establecidos por las políticas operativas. Dichas inconsistencias impactan directamente en las estadísticas de transaccionalidad, generando desviaciones significativas en los reportes de comportamiento financiero.

- Adicionalmente, se observó que las operaciones involucradas fueron realizadas por el mismo usuario y en el mismo ATM, se procedera a eliminar el dato atipico, adicional se procedera a revisar si existen otros valores inusuales.

In [14]:
# Definir umbral 
umbral = 20000
# Filtrar las filas con los valores inflados
df_inflados = df_trx_emisor[df_trx_emisor['DE6_Monto_Dolar']> umbral]
df_inflados.head(20)


,DE32_Cod_Adquiriente,DE42_Cod_Comercio,DE49_Cod_Moneda,DE61_13_Cod_Pais,ID_Usuario,DE13_Fecha,Id_Trx,Comercio_Alias,DE18_MCC,DE6_Monto_Dolar,DE4_Monto_Local,DE22_Modo_Entrada,Aut,DE25_Punto_Entrada,Regla,DE39_Respuesta_ISO,Id_Terminal_Hash,MTI,Cod_Resp,entry_mode,Desc_entry_mode,canal,Registro,IIN,ID_Tarjeta_Hash,Estado,BIN_TC_TD,dia_semana,Respuesta,rango_monto,year,mes,day,hora,mes_year,trimestre,Fraude,franja_horaria,Regla_nrt
1232051,ADQ_7b28194e,COM_c9ce7f72,840,218,a78d03d2d6d2,2025-02-07 09:00:47,1486310,LC_9540c1090d,6011,2211089408.00,2211089408.00,0510,0,2252PRO11010P,0,51,TERM_fca396848a,0200,4007,05,Chip,CP,Compra,000001,e7591fbbaddb,None,TD,Friday,NO_AUTORIZADO,+1000,2025,2,7,9,2025-02,2025Q1,0,mañana,NO_ALERTA
1241525,ADQ_7b28194e,COM_c9ce7f72,840,218,a78d03d2d6d2,2025-02-11 12:28:17,1497210,LC_9540c1090d,6011,2211089408.00,2211089408.00,0510,0,2252PRO11010P,0,51,TERM_fca396848a,0200,4007,05,Chip,CP,Compra,000001,e7591fbbaddb,None,TD,Tuesday,NO_AUTORIZADO,+1000,2025,2,11,12,2025-02,2025Q1,0,tarde,NO_ALERTA
1241568,ADQ_7b28194e,COM_c9ce7f72,840,218,a78d03d2d6d2,2025-02-11 12:48:06,1497260,LC_9540c1090d,6011,2211089408.00,2211089408.00,0510,0,2252PRO11010P,0,51,TERM_fca396848a,0200,4007,05,Chip,CP,Compra,000001,e7591fbbaddb,None,TD,Tuesday,NO_AUTORIZADO,+1000,2025,2,11,12,2025-02,2025Q1,0,tarde,NO_ALERTA



Eliminacion de valores atipicos

- Se identificaron tres registros con montos extremadamente altos en la columna `DE6_Monto_Dolar`, considerados atípicos y poco representativos del comportamiento real de las transacciones.  
- Para depurar el dataset, se establece un **umbral máximo razonable de 20000**. Si bien los montos típicos de las tarjetas suelen ser mucho menores, 
- Este umbral permite eliminar los datos claramente anómalos sin afectar significativamente la distribución general.
- Al eliminar este valor atipico se evidencia que el valor maximo en monto dolar es de 17150, lo cual son datos mas acercados a la realidad.


In [15]:
df_trx_emisor = df_trx_emisor[df_trx_emisor['DE6_Monto_Dolar'] < umbral]
df_trx_emisor.describe(include=[np.number])


,DE6_Monto_Dolar,DE4_Monto_Local,year,mes,day,hora,Fraude
count,1463837.00,1463837.00,1463837.00,1463837.00,1463837.00,1463837.00,1463837.00
mean,40.33,581.42,2023.99,6.61,15.28,13.83,0.01
std,98.62,29459.37,0.70,3.56,8.74,4.88,0.08
min,0.00,0.00,2023.00,1.00,1.00,0.00,0.00
25%,5.15,5.00,2023.00,3.00,8.00,11.00,0.00
50%,15.42,16.00,2024.00,6.00,15.00,14.00,0.00
75%,45.00,48.00,2024.00,10.00,23.00,18.00,0.00
max,17150.00,15750000.00,2025.00,12.00,31.00,23.00,1.00


In [16]:

# Guardar DataFrame limpio como Parquet
df_trx_emisor.to_parquet(r'C:\Users\xavie\Documents\PROYECTOS\fraud-detection-ml\data\processed\01_df_limpieza.parquet')
